import the all libraries

In [4]:
import pandas as pd
import os
from tqdm import tqdm
import numpy as np
import re
import nltk
nltk.download('omw-1.4')
import spacy
import string
from nltk.corpus import stopwords
from collections import Counter
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize,sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import nltk
from nltk.stem import PorterStemmer

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


here download the nltk(natural language toolkit ) library

In [5]:
nltk.download('stopwords')
stop_words=set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


here import the libraries for the graph plotting

In [6]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

In [7]:
from sklearn.model_selection import train_test_split,cross_validate
from sklearn.metrics import accuracy_score,roc_auc_score,f1_score,classification_report,precision_score, recall_score
from mlxtend.plotting import plot_confusion_matrix

In [8]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from xgboost import XGBClassifier

In [10]:
!pip install pad_sequences

  Preparing metadata (setup.py) ... done
  Created wheel for pad_sequences: filename=pad_sequences-0.6.1-py3-none-any.whl size=10199 sha256=e4eea39bc1b96d80316a59dafb80aeb34e794df25b9153dfea94fd7e73d77898
  Stored in directory: /root/.cache/pip/wheels/48/9d/22/0a6305b87a9cc46ccc032060a041c3b59f39ac462f7358997e
Successfully built pad_sequences


In [11]:
!pip install keras_preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 687.5 kB/s eta 0:00:00


In [12]:
!pip install keras_tuner


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 3.1 MB/s eta 0:00:00


In [13]:
!pip install tensorflow_addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 5.8 MB/s eta 0:00:00


here import the tensorflow and keras function

In [14]:
import tensorflow as tf
import keras
from keras.preprocessing.text import one_hot,Tokenizer, text_to_word_sequence
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Dense , Flatten ,Embedding,Input,CuDNNLSTM,LSTM, BatchNormalization, Dropout, InputLayer, ReLU
from keras.initializers import Constant
from keras.callbacks import EarlyStopping
import keras_tuner as kt
import tensorflow_addons as tfa

F1_Score_tf = tfa.metrics.F1Score(num_classes=1,average="weighted")
auc_roc_metric_ROC = keras.metrics.AUC(curve='ROC')

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [15]:
import gensim
from gensim.models import Word2Vec,KeyedVectors
from nltk.tokenize import sent_tokenize, word_tokenize

read the dataset

In [16]:
data = pd.read_csv("/content/twitter_parsed_dataset.csv.zip")
data.dropna(inplace=True)
data.reset_index(inplace=True,drop=True)
data

,index,id,Text,Annotation,oh_label
0,5.74948705591165E+017,5.74948705591165E+017,@halalflaws @biebervalue @greenlinerzjm I read...,none,0.0
1,5.71917888690393E+017,5.71917888690393E+017,@ShreyaBafna3 Now you idiots claim that people...,none,0.0
2,3.90255841338601E+017,3.90255841338601E+017,"RT @Mooseoftorment Call me sexist, but when I ...",sexism,1.0
3,5.68208850655916E+017,5.68208850655916E+017,"@g0ssipsquirrelx Wrong, ISIS follows the examp...",racism,1.0
4,5.75596338802373E+017,5.75596338802373E+017,#mkr No No No No No No,none,0.0
...,...,...,...,...,...
16843,5.75606766236475E+017,5.75606766236475E+017,"Feeling so sorry for the girls, they should be...",none,0.0
16844,5.72333822886326E+017,5.72333822886326E+017,#MKR 'pretty good dishes we're happy with' - O...,none,0.0
16845,5.72326950057845E+017,5.72326950057845E+017,RT @colonelkickhead: Deconstructed lemon tart!...,none,0.0
16846,5.74799612642357E+017,5.74799612642357E+017,@versacezaynx @nyazpolitics @greenlinerzjm You...,none,0.0


drop the three columns

In [17]:
data.drop(columns=['index','id','Annotation'],inplace=True)
data

,Text,oh_label
0,@halalflaws @biebervalue @greenlinerzjm I read...,0.0
1,@ShreyaBafna3 Now you idiots claim that people...,0.0
2,"RT @Mooseoftorment Call me sexist, but when I ...",1.0
3,"@g0ssipsquirrelx Wrong, ISIS follows the examp...",1.0
4,#mkr No No No No No No,0.0
...,...,...
16843,"Feeling so sorry for the girls, they should be...",0.0
16844,#MKR 'pretty good dishes we're happy with' - O...,0.0
16845,RT @colonelkickhead: Deconstructed lemon tart!...,0.0
16846,@versacezaynx @nyazpolitics @greenlinerzjm You...,0.0


here define the many function which have need in data preprocessing

In [18]:
def count_chars(text):
  return len(text)
def count_words(text):
  return len(text.split())
def count_capital_chars(text):
  count=0
  for i in text:
    if i.isupper():
      count+=1
    return count
def count_capital_words(text):
    return sum(map(str.isupper,text.split()))
def count_punctuations(text):
    punctuations='!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    d=0
    for i in punctuations:
        d = d + text.count(i)
    return d
def count_words_in_quotes(text):
    x = re.findall("\'.\'|\".\"", text)
    count=0
    if x is None:
        return 0
    else:
        for i in x:
            t=i[1:-1]
            count+=count_words(t)
        return count
def count_sent(text):
    return len(nltk.sent_tokenize(text))
def avg_sent_len(word_cnt,sent_cnt):
    return word_cnt/sent_cnt
def count_unique_words(text):
    return len(set(text.split()))
def words_vs_unique(words,unique):
    return unique/words
def count_htags(text):
    x = re.findall(r'(\#\w[A-Za-z0-9]*)', text)
    return len(x)
def count_mentions(text):
    x = re.findall(r'(\@\w[A-Za-z0-9]*)', text)
    return len(x)
def count_stopwords(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    stopwords_x = [w for w in word_tokens if w in stop_words]
    return len(stopwords_x)
def stopwords_vs_words(stopwords_cnt,text):
    return stopwords_cnt/len(word_tokenize(text))

In [19]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [20]:
data['char_count'] = data["Text"].apply(lambda x:count_chars(str(x)))
data['word_count'] = data["Text"].apply(lambda x:count_words(str(x)))
data['sent_count'] = data["Text"].apply(lambda x:count_sent(str(x)))
data['capital_char_count'] = data["Text"].apply(lambda x:count_capital_chars(str(x)))
data['capital_word_count'] = data["Text"].apply(lambda x:count_capital_words(str(x)))
data['quoted_word_count'] = data["Text"].apply(lambda x:count_words_in_quotes(str(x)))
data['stopword_count'] = data["Text"].apply(lambda x:count_stopwords(str(x)))
data['unique_word_count'] = data["Text"].apply(lambda x:count_unique_words(str(x)))
data['htag_count'] = data["Text"].apply(lambda x:count_htags(str(x)))
data['mention_count'] = data["Text"].apply(lambda x:count_mentions(str(x)))
data['punct_count'] = data["Text"].apply(lambda x:count_punctuations(str(x)))
data['avg_wordlength']=data['char_count']/data['word_count']
data['avg_sentlength']=data['word_count']/data['sent_count']
data['unique_vs_words']=data['unique_word_count']/data['word_count']
data['stopwords_vs_words']=data['stopword_count']/data['word_count']

here given data set have emoji and punctuation mark

In [21]:
data["Text"][145:155].values

array(['@chilblane everyone should be jealous of my coffee maker. our love is pure.',
       '@obsurfer84 @teenage Let people read the Quran and see that it comes from hate. http://t.co/cuQDA0n32q',
       '#MKR2015 #MKR why do they keep saying that they are so pretty',
       'I HATE KAT AND ANDRE ITS A COOKING COMPETITION STOP VOTING STRATEGICALLY YOU SOGGY BREADS ROLLS #MKR',
       '@JillWetzler I ❤️ u',
       "@discerningmumin We didn't overturn elections in either place, the people did, and the people have that right, as long as they elect new ppl",
       "I'm so sick of Kat n Andre scraping through by their teeth. KICK THEM OFF THE SHOW! #MKR",
       '@LifeInKhilafah When Daesh made children into killers, committing genocide against minorities, and slavery, they wrote their own obituary.',
       "Call me crazy, but I like my food constructed. I wouldn't pay for a build it yourself dessert #MKR",
       '@gamer456ism @femfreq @StephenAtHome "you\'re"'], dtype=object)

In [22]:
data["Text"][2550:2560].values

array(['RT @Kasparov63: Moral relativism that says there is not good or evil, only "different", is how evil wins. By convincing us there is nothing…',
       "RT @DustyPee: If #cuntandandre make it through, I'll... I don't know what I will do. #mkr",
       "Have a feeling u'd find s'thing 2 complain about. @Hheath10 wouldn't care if she was just an average ref but she got it all wrong #notsexist",
       '@ChristophHeer52 @miskelayla Some day I will go out of my way to piss on his grave.',
       'Argh another round of instant restaurants....over it!!!! #mkr',
       'Oh thank god Sheri &amp; Emilie #MKR 😉',
       "RT @Omg_Mitch: I'm not sexist, but sexist jokes are funny",
       'Their restaurant is so TACKY #mkr',
       "RT @JeffreyGoldberg: On Yemen's last Jews: https://t.co/0f2OLVVIng",
       '@Dabiq_Warrior @Totenleserin Daesh threw a major offensive Friday to show they were still strong.  They only managed to show their weakness.'],
      dtype=object)

here apply the defined function for the removing of emoji and emoticon

In [23]:
import pickle
with open('Emoji_Dict.p', 'rb') as fp:
    Emoji_Dict = pickle.load(fp)
Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

with open('Emoticon_Dict.p', 'rb') as fp:
    Emoticon_Dict = pickle.load(fp)
Emoticon_Dict = {v: k for k, v in Emoticon_Dict.items()}

def convert_emojis_to_word(text):
    for emot in Emoji_Dict:
        text = re.sub(r'('+emot+')', "_".join(Emoji_Dict[emot].replace(",","").replace(":","").split()), text)
    return text

def convert_emoticons(text):
    for emot in Emoticon_Dict:
        text = re.sub(u'('+emot+')', "_".join(Emoticon_Dict[emot].replace(",","").split()), text)
    return text

here tqdm use for the time of iteration

In [24]:
from tqdm import tqdm
tqdm.pandas(desc="my bar!")

column = "Text"
data[column] = data[column].progress_apply(lambda text: convert_emoticons(str(text)))

my bar!: 100%|██████████| 16848/16848 [00:02<00:00, 6814.94it/s]


In [31]:
data[column] = data[column].progress_apply(lambda text: convert_emojis_to_word(str(text)))

my bar!: 100%|██████████| 16848/16848 [36:07<00:00,  7.77it/s]


preprocessed data set without emoji

In [33]:
print( data["Text"][145:155].values )
print( data["Text"][2550:2560].values )

['@chilblane everyone should be jealous of my coffee maker. our love is pure.'
 '@obsurfer84 @teenage Let people read the Quran and see that it comes from hate. http://t.co/cuQDA0n32q'
 '#MKR2015 #MKR why do they keep saying that they are so pretty'
 'I HATE KAT AND ANDRE ITS A COOKING COMPETITION STOP VOTING STRATEGICALLY YOU SOGGY BREADS ROLLS #MKR'
 '@JillWetzler I red_heart️ u'
 "@discerningmumin We didn't overturn elections in either place, the people did, and the people have that right, as long as they elect new ppl"
 "I'm so sick of Kat n Andre scraping through by their teeth. KICK THEM OFF THE SHOW! #MKR"
 '@LifeInKhilafah When Daesh made children into killers, committing genocide against minorities, and slavery, they wrote their own obituary.'
 "Call me crazy, but I like my food constructed. I wouldn't pay for a build it yourself dessert #MKR"
 '@gamer456ism @femfreq @StephenAtHome "you\'re"']
['RT @Kasparov63: Moral relativism that says there is not good or evil, only "differ

In [34]:
data_copy=data.copy()
data_copy

,Text,oh_label,char_count,word_count,sent_count,capital_char_count,capital_word_count,quoted_word_count,stopword_count,unique_word_count,htag_count,mention_count,punct_count,avg_wordlength,avg_sentlength,unique_vs_words,stopwords_vs_words
0,@halalflaws @biebervalue @greenlinerzjm I read...,0.0,140,17,3,0,1,0,4,16,0,3,11,8.235294,5.666667,0.941176,0.235294
1,@ShreyaBafna3 Now you idiots claim that people...,0.0,140,23,2,0,0,0,9,21,0,1,3,6.086957,11.500000,0.913043,0.391304
2,"RT @Mooseoftorment Call me sexist, but when I ...",1.0,91,19,1,1,2,0,7,18,0,1,4,4.789474,19.000000,0.947368,0.368421
3,"@g0ssipsquirrelx Wrong, ISIS follows the examp...",1.0,83,12,1,0,1,0,4,11,0,1,3,6.916667,12.000000,0.916667,0.333333
4,#mkr No No No No No No,0.0,22,7,1,0,0,0,0,2,1,0,1,3.142857,7.000000,0.285714,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16843,"Feeling so sorry for the girls, they should be...",0.0,95,19,1,1,1,0,10,16,1,0,2,5.000000,19.000000,0.842105,0.526316
16844,#MKR 'pretty good dishes we're happy with' - O...,0.0,85,16,1,0,2,0,4,16,1,0,8,5.312500,16.000000,1.000000,0.250000
16845,RT @colonelkickhead: Deconstructed lemon tart!...,0.0,140,18,1,1,2,0,4,18,1,1,9,7.777778,18.000000,1.000000,0.222222
16846,@versacezaynx @nyazpolitics @greenlinerzjm You...,0.0,82,11,2,0,0,0,4,11,0,3,5,7.454545,5.500000,1.000000,0.363636


In [35]:
data=data_copy.copy()
data

,Text,oh_label,char_count,word_count,sent_count,capital_char_count,capital_word_count,quoted_word_count,stopword_count,unique_word_count,htag_count,mention_count,punct_count,avg_wordlength,avg_sentlength,unique_vs_words,stopwords_vs_words
0,@halalflaws @biebervalue @greenlinerzjm I read...,0.0,140,17,3,0,1,0,4,16,0,3,11,8.235294,5.666667,0.941176,0.235294
1,@ShreyaBafna3 Now you idiots claim that people...,0.0,140,23,2,0,0,0,9,21,0,1,3,6.086957,11.500000,0.913043,0.391304
2,"RT @Mooseoftorment Call me sexist, but when I ...",1.0,91,19,1,1,2,0,7,18,0,1,4,4.789474,19.000000,0.947368,0.368421
3,"@g0ssipsquirrelx Wrong, ISIS follows the examp...",1.0,83,12,1,0,1,0,4,11,0,1,3,6.916667,12.000000,0.916667,0.333333
4,#mkr No No No No No No,0.0,22,7,1,0,0,0,0,2,1,0,1,3.142857,7.000000,0.285714,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16843,"Feeling so sorry for the girls, they should be...",0.0,95,19,1,1,1,0,10,16,1,0,2,5.000000,19.000000,0.842105,0.526316
16844,#MKR 'pretty good dishes we're happy with' - O...,0.0,85,16,1,0,2,0,4,16,1,0,8,5.312500,16.000000,1.000000,0.250000
16845,RT @colonelkickhead: Deconstructed lemon tart!...,0.0,140,18,1,1,2,0,4,18,1,1,9,7.777778,18.000000,1.000000,0.222222
16846,@versacezaynx @nyazpolitics @greenlinerzjm You...,0.0,82,11,2,0,0,0,4,11,0,3,5,7.454545,5.500000,1.000000,0.363636


here work on removing of the url and htm tags

In [36]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

data[column] = data[column].apply(lambda text: remove_urls(str(text)))
data[column] = data[column].apply(lambda text: remove_html(str(text)))
data

,Text,oh_label,char_count,word_count,sent_count,capital_char_count,capital_word_count,quoted_word_count,stopword_count,unique_word_count,htag_count,mention_count,punct_count,avg_wordlength,avg_sentlength,unique_vs_words,stopwords_vs_words
0,@halalflaws @biebervalue @greenlinerzjm I read...,0.0,140,17,3,0,1,0,4,16,0,3,11,8.235294,5.666667,0.941176,0.235294
1,@ShreyaBafna3 Now you idiots claim that people...,0.0,140,23,2,0,0,0,9,21,0,1,3,6.086957,11.500000,0.913043,0.391304
2,"RT @Mooseoftorment Call me sexist, but when I ...",1.0,91,19,1,1,2,0,7,18,0,1,4,4.789474,19.000000,0.947368,0.368421
3,"@g0ssipsquirrelx Wrong, ISIS follows the examp...",1.0,83,12,1,0,1,0,4,11,0,1,3,6.916667,12.000000,0.916667,0.333333
4,#mkr No No No No No No,0.0,22,7,1,0,0,0,0,2,1,0,1,3.142857,7.000000,0.285714,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16843,"Feeling so sorry for the girls, they should be...",0.0,95,19,1,1,1,0,10,16,1,0,2,5.000000,19.000000,0.842105,0.526316
16844,#MKR 'pretty good dishes we're happy with' - O...,0.0,85,16,1,0,2,0,4,16,1,0,8,5.312500,16.000000,1.000000,0.250000
16845,RT @colonelkickhead: Deconstructed lemon tart!...,0.0,140,18,1,1,2,0,4,18,1,1,9,7.777778,18.000000,1.000000,0.222222
16846,@versacezaynx @nyazpolitics @greenlinerzjm You...,0.0,82,11,2,0,0,0,4,11,0,3,5,7.454545,5.500000,1.000000,0.363636


removing the number

In [37]:
def remove_numbers(text):
    number_pattern = r'\d+'
    without_number = re.sub(pattern=number_pattern, repl=" ", string=text)
    return without_number

data[column] = data[column].progress_apply(lambda text: remove_numbers(str(text)))
data

my bar!: 100%|██████████| 16848/16848 [00:00<00:00, 106478.67it/s]


,Text,oh_label,char_count,word_count,sent_count,capital_char_count,capital_word_count,quoted_word_count,stopword_count,unique_word_count,htag_count,mention_count,punct_count,avg_wordlength,avg_sentlength,unique_vs_words,stopwords_vs_words
0,@halalflaws @biebervalue @greenlinerzjm I read...,0.0,140,17,3,0,1,0,4,16,0,3,11,8.235294,5.666667,0.941176,0.235294
1,@ShreyaBafna Now you idiots claim that people...,0.0,140,23,2,0,0,0,9,21,0,1,3,6.086957,11.500000,0.913043,0.391304
2,"RT @Mooseoftorment Call me sexist, but when I ...",1.0,91,19,1,1,2,0,7,18,0,1,4,4.789474,19.000000,0.947368,0.368421
3,"@g ssipsquirrelx Wrong, ISIS follows the examp...",1.0,83,12,1,0,1,0,4,11,0,1,3,6.916667,12.000000,0.916667,0.333333
4,#mkr No No No No No No,0.0,22,7,1,0,0,0,0,2,1,0,1,3.142857,7.000000,0.285714,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16843,"Feeling so sorry for the girls, they should be...",0.0,95,19,1,1,1,0,10,16,1,0,2,5.000000,19.000000,0.842105,0.526316
16844,#MKR 'pretty good dishes we're happy with' - O...,0.0,85,16,1,0,2,0,4,16,1,0,8,5.312500,16.000000,1.000000,0.250000
16845,RT @colonelkickhead: Deconstructed lemon tart!...,0.0,140,18,1,1,2,0,4,18,1,1,9,7.777778,18.000000,1.000000,0.222222
16846,@versacezaynx @nyazpolitics @greenlinerzjm You...,0.0,82,11,2,0,0,0,4,11,0,3,5,7.454545,5.500000,1.000000,0.363636


make all data set in lowercase

In [38]:
def make_lowercase(data,column):
    data[column] = data[column].str.lower()
    return data
data = make_lowercase(data,column='Text')

In [39]:
from textblob import TextBlob
data["sentiment"] = data["Text"].progress_apply(lambda x: TextBlob(x).sentiment.polarity)
data

my bar!: 100%|██████████| 16848/16848 [00:05<00:00, 3091.31it/s]


,Text,oh_label,char_count,word_count,sent_count,capital_char_count,capital_word_count,quoted_word_count,stopword_count,unique_word_count,htag_count,mention_count,punct_count,avg_wordlength,avg_sentlength,unique_vs_words,stopwords_vs_words,sentiment
0,@halalflaws @biebervalue @greenlinerzjm i read...,0.0,140,17,3,0,1,0,4,16,0,3,11,8.235294,5.666667,0.941176,0.235294,0.000000
1,@shreyabafna now you idiots claim that people...,0.0,140,23,2,0,0,0,9,21,0,1,3,6.086957,11.500000,0.913043,0.391304,-0.183333
2,"rt @mooseoftorment call me sexist, but when i ...",1.0,91,19,1,1,2,0,7,18,0,1,4,4.789474,19.000000,0.947368,0.368421,0.000000
3,"@g ssipsquirrelx wrong, isis follows the examp...",1.0,83,12,1,0,1,0,4,11,0,1,3,6.916667,12.000000,0.916667,0.333333,-0.125000
4,#mkr no no no no no no,0.0,22,7,1,0,0,0,0,2,1,0,1,3.142857,7.000000,0.285714,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16843,"feeling so sorry for the girls, they should be...",0.0,95,19,1,1,1,0,10,16,1,0,2,5.000000,19.000000,0.842105,0.526316,0.000000
16844,#mkr 'pretty good dishes we're happy with' - o...,0.0,85,16,1,0,2,0,4,16,1,0,8,5.312500,16.000000,1.000000,0.250000,0.562500
16845,rt @colonelkickhead: deconstructed lemon tart!...,0.0,140,18,1,1,2,0,4,18,1,1,9,7.777778,18.000000,1.000000,0.222222,0.000000
16846,@versacezaynx @nyazpolitics @greenlinerzjm you...,0.0,82,11,2,0,0,0,4,11,0,3,5,7.454545,5.500000,1.000000,0.363636,-0.800000


In [40]:
no_words = set(['no',"wasn't","has't","didn't","hadn't","mightn't","needn't","won't","weren't","doesn't","mustn't","shouldn't","haven't","doesn","hadn","didn't","not","needn","aren't","isn","aren","isn't","wasn","hasn't","didn","shouldn","don't","couldn't","wouldn't","weren","hasn",""])
stop_words_n = set(stopwords.words('english'))-no_words

def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop_words_n])

data["Text"] = data["Text"].progress_apply(lambda text: remove_stopwords(text))

my bar!: 100%|██████████| 16848/16848 [00:00<00:00, 208221.02it/s]


In [41]:
def remove_AtSign_tags(data,column):
    data[column] = data[column].apply( lambda text: re.sub('\@[a-zA-Z]+','',text) )
    return data

data = remove_AtSign_tags(data,column='Text')

In [42]:
def remove_puntuations(data,column):
    data[column] = data[column].progress_apply(lambda text: str(text).translate(str.maketrans('','',string.punctuation)))
    return data
data = remove_puntuations(data,"Text")

my bar!: 100%|██████████| 16848/16848 [00:00<00:00, 157645.71it/s]


In [43]:
def remove_single_char(text):
    single_char_pattern = r'\s+[a-zA-Z]\s+'
    without_sc = re.sub(pattern=single_char_pattern, repl=" ", string=text)
    return without_sc
data[column] = data[column].progress_apply(lambda text: remove_single_char(str(text)))

my bar!: 100%|██████████| 16848/16848 [00:00<00:00, 230736.64it/s]


In [44]:
def remove_most_frequent_words(data,column,frequent_words_count):
    all_text = ' '.join(data[column].tolist())
    cnt = Counter(all_text.split())
    most_frequent_words = cnt.most_common(frequent_words_count)
    data[column] = data[column].progress_apply(lambda text: " ".join(list(set(text.split()) - set(most_frequent_words))))
    return data
def remove_rare_words(data,column,rare_words_count):
    all_text = ' '.join(data[column].tolist())
    cnt = Counter(all_text.split())
    most_frequent_words = cnt.most_common()
    rare_words = most_frequent_words[len(most_frequent_words)-rare_words_count:]
    rare_words = set(dict(rare_words).keys())
    data[column] = data[column].progress_apply(lambda text: " ".join(list(set(text.split()) - rare_words)))
    return data
data = remove_most_frequent_words(data,column='Text',frequent_words_count=20)
data = remove_rare_words(data,column="Text",rare_words_count=1000)

my bar!: 100%|██████████| 16848/16848 [00:00<00:00, 245579.11it/s]


In [45]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [46]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [47]:
column = "Text"
lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])
data[column] = data[column].progress_apply(lambda text: lemmatize_words(text))
data

my bar!: 100%|██████████| 16848/16848 [00:13<00:00, 1228.35it/s]


,Text,oh_label,char_count,word_count,sent_count,capital_char_count,capital_word_count,quoted_word_count,stopword_count,unique_word_count,htag_count,mention_count,punct_count,avg_wordlength,avg_sentlength,unique_vs_words,stopwords_vs_words,sentiment
0,mean read history islamic slavery change conte...,0.0,140,17,3,0,1,0,4,16,0,3,11,8.235294,5.666667,0.941176,0.235294,0.000000
1,stop people claim become brain dead make islam...,0.0,140,23,2,0,0,0,9,21,0,1,3,6.086957,11.500000,0.913043,0.391304,-0.183333
2,guy place rather auto go call talk id sexist rt,1.0,91,19,1,1,2,0,7,18,0,1,4,4.789474,19.000000,0.947368,0.368421,0.000000
3,example isi wrong quran exactly mohammed follo...,1.0,83,12,1,0,1,0,4,11,0,1,3,6.916667,12.000000,0.916667,0.333333,-0.125000
4,mkr no,0.0,22,7,1,0,0,0,0,2,1,0,1,3.142857,7.000000,0.285714,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16843,home sorry safe andre kat go feel mkr girl,0.0,95,19,1,1,1,0,10,16,1,0,2,5.000000,19.000000,0.842105,0.526316,0.000000
16844,never ok place happy dish with pretty well eat...,0.0,85,16,1,0,2,0,4,16,1,0,8,5.312500,16.000000,1.000000,0.250000,0.562500
16845,please one deconstruct someone lemon without g...,0.0,140,18,1,1,2,0,4,18,1,1,9,7.777778,18.000000,1.000000,0.222222,0.000000
16846,talk to block stupid,0.0,82,11,2,0,0,0,4,11,0,3,5,7.454545,5.500000,1.000000,0.363636,-0.800000


In [48]:
data['oh_label'] = data['oh_label'].astype(int)
index = []
for i in tqdm(list(data.index)):
    count = len(data['Text'][i].split())
    if (count<=3) & (data['oh_label'][i]==0):
        index.append(i)
data.drop(index=index,inplace=True)

100%|██████████| 16848/16848 [00:00<00:00, 64193.92it/s]


In [49]:
data.to_csv("preprocessed_text_twitter_parsed_dataset.csv")

In [50]:
data = pd.read_csv("preprocessed_text_twitter_parsed_dataset.csv")
data.drop(columns="Unnamed: 0",inplace=True)
data.dropna(axis="index",inplace=True)
data

,Text,oh_label,char_count,word_count,sent_count,capital_char_count,capital_word_count,quoted_word_count,stopword_count,unique_word_count,htag_count,mention_count,punct_count,avg_wordlength,avg_sentlength,unique_vs_words,stopwords_vs_words,sentiment
0,mean read history islamic slavery change conte...,0,140,17,3,0,1,0,4,16,0,3,11,8.235294,5.666667,0.941176,0.235294,0.000000
1,stop people claim become brain dead make islam...,0,140,23,2,0,0,0,9,21,0,1,3,6.086957,11.500000,0.913043,0.391304,-0.183333
2,guy place rather auto go call talk id sexist rt,1,91,19,1,1,2,0,7,18,0,1,4,4.789474,19.000000,0.947368,0.368421,0.000000
3,example isi wrong quran exactly mohammed follo...,1,83,12,1,0,1,0,4,11,0,1,3,6.916667,12.000000,0.916667,0.333333,-0.125000
4,torture … yearold rap release preacher five de...,0,140,19,1,1,1,0,6,19,0,1,11,7.368421,19.000000,1.000000,0.315789,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15116,home sorry safe andre kat go feel mkr girl,0,95,19,1,1,1,0,10,16,1,0,2,5.000000,19.000000,0.842105,0.526316,0.000000
15117,never ok place happy dish with pretty well eat...,0,85,16,1,0,2,0,4,16,1,0,8,5.312500,16.000000,1.000000,0.250000,0.562500
15118,please one deconstruct someone lemon without g...,0,140,18,1,1,2,0,4,18,1,1,9,7.777778,18.000000,1.000000,0.222222,0.000000
15119,talk to block stupid,0,82,11,2,0,0,0,4,11,0,3,5,7.454545,5.500000,1.000000,0.363636,-0.800000


In [51]:
def Plot_Loss_Accuracy( history, epochs ):
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    auc = history.history['accuracy']
    val_auc = history.history['val_accuracy']

    epochs = np.linspace(1,epochs,epochs)
    epochs
    fig = go.Figure(data=go.Scatter( x=epochs, y=loss, name='Loss' ))
    fig.add_trace( go.Scatter( x=epochs, y=val_loss, name='Validation Lass' ) )
    fig.layout.update( xaxis_title = 'No. of Epochs',
                       yaxis_title = "Loss",
                       legend=dict( yanchor="bottom", y=0.01, xanchor="right", x=0.99),
                       width=750, height=650,
                       font=dict(size=18))
    fig.show()
    fig = go.Figure(data=go.Scatter( x=epochs, y=auc, name='Accuracy Score' ))
    fig.add_trace( go.Scatter( x=epochs, y=val_auc, name='Validation Accuracy Score' ) )
    fig.layout.update( xaxis_title = 'No. of Epochs',
                       yaxis_title = "Accuracy Scores",
                       legend=dict( yanchor="bottom", y=0.01, xanchor="right", x=0.99),
                       width=750, height=650,
                       font=dict(size=18))
    fig.show()

In [52]:
def Evaluation(model,X_train,X_test,y_train,y_test,hypertuning=False):

  plt.figure(  figsize=(12,6) )
  y_pred = model.predict(X_train)
  y_pred_proba = model.predict_proba(X_train)

  accuracy_train = accuracy_score( y_train, y_pred )
  precision_train = precision_score( y_train, y_pred )
  recall_train = recall_score(y_train, y_pred)
  F1_score_train = f1_score(y_train, y_pred)
  roc_auc_train = roc_auc_score(y_train, y_pred_proba[:,1])
  y_pred = model.predict(X_test)
  y_pred_proba = model.predict_proba(X_test)

  accuracy_test = accuracy_score( y_test, y_pred )
  precision_test = precision_score( y_test, y_pred )
  recall_test = recall_score(y_test, y_pred)
  F1_score_test = f1_score(y_test, y_pred)
  roc_auc_test = roc_auc_score(y_test, y_pred_proba[:,1])
  return  accuracy_train, precision_train, recall_train, F1_score_train, roc_auc_train, accuracy_test, precision_test, recall_test, F1_score_test, roc_auc_test

In [53]:
def apply_models_with_default_paramters(X_train,X_test,y_train,y_test):
  models_default = [ XGBClassifier(),
                     GaussianNB(),
                     RandomForestClassifier(),
                     DecisionTreeClassifier(),
                     AdaBoostClassifier(),
                     ]
  F1_Score_train = []
  Accuracy_train = []
  Recall_train = []
  Precision_train = []
  ROC_AUC_Score_train = []
  F1_Score_test = []
  Accuracy_test = []
  Recall_test = []
  Precision_test = []
  ROC_AUC_Score_test = []
  Model_Name = []
  for model in models_default:
    Model_Name.append( model )
    model.fit(X_train, y_train)
    accuracy_train, precision_train, recall_train, F1_score_train, roc_auc_train, accuracy_test, precision_test, recall_test, F1_score_test, roc_auc_test = Evaluation(model,X_train,X_test,y_train,y_test,False)

    F1_Score_train.append( F1_score_train )
    Accuracy_train.append( accuracy_train )
    Recall_train.append( recall_train )
    Precision_train.append( precision_train )
    ROC_AUC_Score_train.append( roc_auc_train )

    F1_Score_test.append( F1_score_test )
    Accuracy_test.append( accuracy_test )
    Recall_test.append( recall_test )
    Precision_test.append( precision_test )
    ROC_AUC_Score_test.append( roc_auc_test )

  results = pd.DataFrame()
  results['Model_Name'] = Model_Name

  train_test_f1_score_difference = np.subtract(F1_Score_train,F1_Score_test)
# here check the overfitting and underfitting
  results['F1_Score on Test Set'] = F1_Score_test
  results['Accuracy on Test Set'] = Accuracy_test
  results['Recall on Test Set'] = Recall_test
  results['Precision on Test Set'] = Precision_test
  results['ROC_AUC_Score on Test Set'] = ROC_AUC_Score_test

  results['F1_Score on Train Set'] = F1_Score_train
  results['Accuracy on Train Set'] = Accuracy_train
  results['Recall on Train Set'] = Recall_train
  results['Precision on Train Set'] = Precision_train
  results['ROC_AUC_Score on Train Set'] = ROC_AUC_Score_train

  results['Difference of F1_Score on train and test'] = train_test_f1_score_difference

  results = results.sort_values(by=['F1_Score on Test Set','Difference of F1_Score on train and test'],ascending = [False, True])

  return results

In [54]:
fig = px.pie(values=[sum(data['oh_label']==0),sum(data['oh_label']==1)], names=['no(0)','yes(1)'] , title='Target')
fig.update_layout(width=300,height=300)
fig.show()

In [55]:
data = pd.read_csv("preprocessed_text_twitter_parsed_dataset.csv")
data.drop(columns="Unnamed: 0",inplace=True)
data.dropna(axis="index",inplace=True)
data
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=["oh_label"]), data[['oh_label']], test_size=0.2, random_state=10, stratify=data['oh_label'])
X_train.reset_index( inplace=True, drop=True ), X_test.reset_index( inplace=True, drop=True ), y_train.reset_index( inplace=True, drop=True ), y_test.reset_index( inplace=True, drop=True )

(None, None, None, None)

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer( lowercase=True, ngram_range=(1, 1), max_features=300,
                               binary=False, stop_words = 'english' )
X_train_text = pd.DataFrame.sparse.from_spmatrix( vectorizer.fit_transform(X_train['Text']) )
X_test_text = pd.DataFrame.sparse.from_spmatrix( vectorizer.transform(X_test['Text']) )
X_train = pd.concat([X_train_text,X_train.drop(columns=['Text'])], axis=1)
X_test = pd.concat([X_test_text,X_test.drop(columns=['Text'])],axis=1)

In [57]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [58]:
model = Sequential()
model.add(Dense(512, input_dim=X_train.shape[1], activation=tf.keras.layers.ReLU()))
model.add(Dense(256, activation=tf.keras.layers.ReLU()))
model.add(Dense(64, activation=tf.keras.layers.ReLU()))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy' , optimizer='adam', metrics=['accuracy', F1_Score_tf])
history = model.fit(X_train, y_train , validation_data=(X_test,y_test), epochs=20, batch_size=32)
Plot_Loss_Accuracy( history, epochs=20 )

Epoch 1/20
378/378 [==============================] - 6s 11ms/step - loss: 0.6819 - accuracy: 0.6500 - f1_score: 0.5224 - val_loss: 0.5740 - val_accuracy: 0.6812 - val_f1_score: 0.5224
Epoch 2/20
378/378 [==============================] - 3s 8ms/step - loss: 0.5190 - accuracy: 0.7500 - f1_score: 0.5224 - val_loss: 0.4557 - val_accuracy: 0.7960 - val_f1_score: 0.5224
Epoch 3/20
378/378 [==============================] - 3s 8ms/step - loss: 0.4822 - accuracy: 0.7785 - f1_score: 0.5224 - val_loss: 0.4472 - val_accuracy: 0.8032 - val_f1_score: 0.5224
Epoch 4/20
378/378 [==============================] - 3s 8ms/step - loss: 0.4584 - accuracy: 0.7893 - f1_score: 0.5224 - val_loss: 0.4367 - val_accuracy: 0.8102 - val_f1_score: 0.5224
Epoch 5/20
378/378 [==============================] - 4s 10ms/step - loss: 0.4439 - accuracy: 0.7990 - f1_score: 0.5224 - val_loss: 0.4400 - val_accuracy: 0.8016 - val_f1_score: 0.5224
Epoch 6/20
378/378 [==============================] - 3s 8ms/step - loss: 0.43

In [59]:
data = pd.read_csv("preprocessed_text_twitter_parsed_dataset.csv")
data.drop(columns="Unnamed: 0",inplace=True)
data.dropna(axis="index",inplace=True)
data

,Text,oh_label,char_count,word_count,sent_count,capital_char_count,capital_word_count,quoted_word_count,stopword_count,unique_word_count,htag_count,mention_count,punct_count,avg_wordlength,avg_sentlength,unique_vs_words,stopwords_vs_words,sentiment
0,mean read history islamic slavery change conte...,0,140,17,3,0,1,0,4,16,0,3,11,8.235294,5.666667,0.941176,0.235294,0.000000
1,stop people claim become brain dead make islam...,0,140,23,2,0,0,0,9,21,0,1,3,6.086957,11.500000,0.913043,0.391304,-0.183333
2,guy place rather auto go call talk id sexist rt,1,91,19,1,1,2,0,7,18,0,1,4,4.789474,19.000000,0.947368,0.368421,0.000000
3,example isi wrong quran exactly mohammed follo...,1,83,12,1,0,1,0,4,11,0,1,3,6.916667,12.000000,0.916667,0.333333,-0.125000
4,torture … yearold rap release preacher five de...,0,140,19,1,1,1,0,6,19,0,1,11,7.368421,19.000000,1.000000,0.315789,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15116,home sorry safe andre kat go feel mkr girl,0,95,19,1,1,1,0,10,16,1,0,2,5.000000,19.000000,0.842105,0.526316,0.000000
15117,never ok place happy dish with pretty well eat...,0,85,16,1,0,2,0,4,16,1,0,8,5.312500,16.000000,1.000000,0.250000,0.562500
15118,please one deconstruct someone lemon without g...,0,140,18,1,1,2,0,4,18,1,1,9,7.777778,18.000000,1.000000,0.222222,0.000000
15119,talk to block stupid,0,82,11,2,0,0,0,4,11,0,3,5,7.454545,5.500000,1.000000,0.363636,-0.800000


In [110]:
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation=tf.keras.layers.LeakyReLU(alpha=0.05)))
model.add(Dense(64, input_dim=X_train.shape[1], activation=tf.keras.layers.LeakyReLU(alpha=0.05)))
model.add(Dense(32, input_dim=X_train.shape[1], activation=tf.keras.layers.LeakyReLU(alpha=0.05)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy' , optimizer='adam', metrics=['accuracy', auc_roc_metric_ROC, F1_Score_tf])
history = model.fit(X_train, y_train , validation_data=(X_test,y_test), epochs=10, batch_size=32)
Plot_Loss_Accuracy( history, epochs=10 )

Epoch 1/10
378/378 [==============================] - 7s 7ms/step - loss: 15.1001 - accuracy: 0.5614 - auc: 0.5541 - f1_score: 0.5031 - val_loss: 6.1248 - val_accuracy: 0.5503 - val_auc: 0.4884 - val_f1_score: 0.4996
Epoch 2/10
378/378 [==============================] - 1s 3ms/step - loss: 4.8758 - accuracy: 0.5647 - auc: 0.5225 - f1_score: 0.5142 - val_loss: 3.7807 - val_accuracy: 0.5618 - val_auc: 0.5395 - val_f1_score: 0.5247
Epoch 3/10
378/378 [==============================] - 1s 4ms/step - loss: 2.8867 - accuracy: 0.5646 - auc: 0.5340 - f1_score: 0.5215 - val_loss: 1.7384 - val_accuracy: 0.5671 - val_auc: 0.5474 - val_f1_score: 0.5233
Epoch 4/10
378/378 [==============================] - 1s 4ms/step - loss: 2.0809 - accuracy: 0.5671 - auc: 0.5248 - f1_score: 0.5210 - val_loss: 2.0224 - val_accuracy: 0.6396 - val_auc: 0.5447 - val_f1_score: 0.5218
Epoch 5/10
378/378 [==============================] - 1s 3ms/step - loss: 1.5341 - accuracy: 0.5704 - auc: 0.5327 - f1_score: 0.5226 - 

In [111]:
data = pd.read_csv("preprocessed_text_twitter_parsed_dataset.csv")
data.drop(columns="Unnamed: 0",inplace=True)
data.dropna(axis="index",inplace=True)
data

,Text,oh_label,char_count,word_count,sent_count,capital_char_count,capital_word_count,quoted_word_count,stopword_count,unique_word_count,htag_count,mention_count,punct_count,avg_wordlength,avg_sentlength,unique_vs_words,stopwords_vs_words,sentiment
0,mean read history islamic slavery change conte...,0,140,17,3,0,1,0,4,16,0,3,11,8.235294,5.666667,0.941176,0.235294,0.000000
1,stop people claim become brain dead make islam...,0,140,23,2,0,0,0,9,21,0,1,3,6.086957,11.500000,0.913043,0.391304,-0.183333
2,guy place rather auto go call talk id sexist rt,1,91,19,1,1,2,0,7,18,0,1,4,4.789474,19.000000,0.947368,0.368421,0.000000
3,example isi wrong quran exactly mohammed follo...,1,83,12,1,0,1,0,4,11,0,1,3,6.916667,12.000000,0.916667,0.333333,-0.125000
4,torture … yearold rap release preacher five de...,0,140,19,1,1,1,0,6,19,0,1,11,7.368421,19.000000,1.000000,0.315789,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15116,home sorry safe andre kat go feel mkr girl,0,95,19,1,1,1,0,10,16,1,0,2,5.000000,19.000000,0.842105,0.526316,0.000000
15117,never ok place happy dish with pretty well eat...,0,85,16,1,0,2,0,4,16,1,0,8,5.312500,16.000000,1.000000,0.250000,0.562500
15118,please one deconstruct someone lemon without g...,0,140,18,1,1,2,0,4,18,1,1,9,7.777778,18.000000,1.000000,0.222222,0.000000
15119,talk to block stupid,0,82,11,2,0,0,0,4,11,0,3,5,7.454545,5.500000,1.000000,0.363636,-0.800000


In [112]:
embed_dim = 300
sentences_list = data['Text'].to_list()
w2v_model = gensim.models.Word2Vec(sentences=sentences_list,window=5,min_count=1,vector_size=embed_dim)
w2v_model.train(sentences_list,epochs=25,total_examples=len(sentences_list))

(3596611, 20618725)

In [113]:
vocab_len = w2v_model.wv.vectors.shape[0]
print("The total number of words are : ", vocab_len )
vocab = w2v_model.wv.key_to_index.keys()

The total number of words are :  77


In [114]:
def document_vector(text,model):
    # remove out-of-vocabulary words
    doc = [word for word in text if word in vocab]
    if len(doc)==0:
        mean = np.nan
    else:
        mean = np.mean(model.wv[doc], axis=0)
    return mean

In [101]:
# Neural network
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation=tf.keras.layers.LeakyReLU(alpha=0.05)))
model.add(Dense(64, input_dim=X_train.shape[1], activation=tf.keras.layers.LeakyReLU(alpha=0.05)))
model.add(Dense(32, input_dim=X_train.shape[1], activation=tf.keras.layers.LeakyReLU(alpha=0.05)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy' , optimizer='adam', metrics=['accuracy', auc_roc_metric_ROC])
history = model.fit(X_train, y_train , validation_data=(X_test,y_test), epochs=10, batch_size=32)

Plot_Loss_Accuracy( history, epochs=10 )

Epoch 1/10
378/378 [==============================] - 3s 4ms/step - loss: 0.6399 - accuracy: 0.6560 - auc: 0.6296 - val_loss: 0.5856 - val_accuracy: 0.6627 - val_auc: 0.7898
Epoch 2/10
378/378 [==============================] - 1s 3ms/step - loss: 0.5222 - accuracy: 0.7498 - auc: 0.7909 - val_loss: 0.4755 - val_accuracy: 0.7864 - val_auc: 0.8448
Epoch 3/10
378/378 [==============================] - 1s 3ms/step - loss: 0.4860 - accuracy: 0.7757 - auc: 0.8217 - val_loss: 0.4446 - val_accuracy: 0.8022 - val_auc: 0.8549
Epoch 4/10
378/378 [==============================] - 2s 4ms/step - loss: 0.4545 - accuracy: 0.7939 - auc: 0.8478 - val_loss: 0.4388 - val_accuracy: 0.8036 - val_auc: 0.8585
Epoch 5/10
378/378 [==============================] - 2s 4ms/step - loss: 0.4557 - accuracy: 0.7926 - auc: 0.8466 - val_loss: 0.4419 - val_accuracy: 0.8072 - val_auc: 0.8614
Epoch 6/10
378/378 [==============================] - 2s 5ms/step - loss: 0.4400 - accuracy: 0.8031 - auc: 0.8582 - val_loss: 0.44

In [102]:
data = pd.read_csv("preprocessed_text_twitter_parsed_dataset.csv")
data.drop(columns="Unnamed: 0",inplace=True)
data.dropna(axis="index",inplace=True)
data

,Text,oh_label,char_count,word_count,sent_count,capital_char_count,capital_word_count,quoted_word_count,stopword_count,unique_word_count,htag_count,mention_count,punct_count,avg_wordlength,avg_sentlength,unique_vs_words,stopwords_vs_words,sentiment
0,mean read history islamic slavery change conte...,0,140,17,3,0,1,0,4,16,0,3,11,8.235294,5.666667,0.941176,0.235294,0.000000
1,stop people claim become brain dead make islam...,0,140,23,2,0,0,0,9,21,0,1,3,6.086957,11.500000,0.913043,0.391304,-0.183333
2,guy place rather auto go call talk id sexist rt,1,91,19,1,1,2,0,7,18,0,1,4,4.789474,19.000000,0.947368,0.368421,0.000000
3,example isi wrong quran exactly mohammed follo...,1,83,12,1,0,1,0,4,11,0,1,3,6.916667,12.000000,0.916667,0.333333,-0.125000
4,torture … yearold rap release preacher five de...,0,140,19,1,1,1,0,6,19,0,1,11,7.368421,19.000000,1.000000,0.315789,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15116,home sorry safe andre kat go feel mkr girl,0,95,19,1,1,1,0,10,16,1,0,2,5.000000,19.000000,0.842105,0.526316,0.000000
15117,never ok place happy dish with pretty well eat...,0,85,16,1,0,2,0,4,16,1,0,8,5.312500,16.000000,1.000000,0.250000,0.562500
15118,please one deconstruct someone lemon without g...,0,140,18,1,1,2,0,4,18,1,1,9,7.777778,18.000000,1.000000,0.222222,0.000000
15119,talk to block stupid,0,82,11,2,0,0,0,4,11,0,3,5,7.454545,5.500000,1.000000,0.363636,-0.800000


In [103]:
tok = Tokenizer()
tok.fit_on_texts(data['Text'])
vocab_size = len(tok.word_index) + 1
print("Vocabulary Size: ", vocab_size )

Vocabulary Size:  14534


In [104]:
def encode_sequence_and_padding(tokenizer, data, column):
    sequences = tokenizer.texts_to_sequences(data[column])
    max_length = int(max([len(vector) for vector in sequences ])/4)
    sequences = pad_sequences( sequences, maxlen=max_length, padding='post')
    return sequences, max_length

sequences, max_length = encode_sequence_and_padding(tok, data=data, column='Text')
sequences = pd.DataFrame(sequences)
print("Maximum length of Sentence: ",max_length)
sequences.to_csv("sequences.csv")

Maximum length of Sentence:  5


In [105]:
X_train, X_test, y_train, y_test = train_test_split(sequences, data['oh_label'], test_size=0.20, random_state = 10, stratify=data['oh_label'])

In [106]:
model = Sequential()
model.add(Embedding( vocab_size, 300, input_length = max_length, mask_zero=True ))
model.add(LSTM(256, return_sequences=True))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(64))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy',auc_roc_metric_ROC, F1_Score_tf])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 5, 300)            4360200   
                                                                 
 lstm (LSTM)                 (None, 5, 256)            570368    
                                                                 
 lstm_1 (LSTM)               (None, 5, 128)            197120    
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense_8 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5,177,161
Trainable params: 5,177,161
Non-trainable params: 0
_________________________________________________________________


In [107]:
epochs=5
batch_size=32
history = model.fit(X_train,y_train,epochs=epochs,batch_size=batch_size,validation_data=(X_test,y_test))
Plot_Loss_Accuracy( history, epochs=epochs )

Epoch 1/5
378/378 [==============================] - 68s 144ms/step - loss: 0.5211 - accuracy: 0.7454 - auc: 0.8043 - f1_score: 0.5224 - val_loss: 0.4775 - val_accuracy: 0.7745 - val_auc: 0.8310 - val_f1_score: 0.5224
Epoch 2/5
378/378 [==============================] - 49s 130ms/step - loss: 0.3699 - accuracy: 0.8423 - auc: 0.9048 - f1_score: 0.5224 - val_loss: 0.4967 - val_accuracy: 0.7745 - val_auc: 0.8203 - val_f1_score: 0.5224
Epoch 3/5
378/378 [==============================] - 54s 144ms/step - loss: 0.2772 - accuracy: 0.8862 - auc: 0.9480 - f1_score: 0.5224 - val_loss: 0.5656 - val_accuracy: 0.7629 - val_auc: 0.8114 - val_f1_score: 0.5224
Epoch 4/5
378/378 [==============================] - 50s 131ms/step - loss: 0.2112 - accuracy: 0.9162 - auc: 0.9697 - f1_score: 0.5224 - val_loss: 0.6811 - val_accuracy: 0.7533 - val_auc: 0.8000 - val_f1_score: 0.5224
Epoch 5/5
378/378 [==============================] - 63s 166ms/step - loss: 0.1577 - accuracy: 0.9363 - auc: 0.9828 - f1_score: 